In [2]:
import polars as pl
import requests


def fetch_news_data(stock, start_date, end_date,turn):
    """
    Makes an API request and returns the response data.
    
    :param endpoint: The API endpoint URL.
    :param headers: Dictionary containing request headers.
    :param params: Dictionary containing query parameters.
    :return: Response JSON as a dictionary or None if the request fails.
    """
    if turn:
        if end_date:
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&time_to={end_date}T0130&limit=1000&apikey=ZRX2N0ETMX8N4GOR"
        else:
            print("65DCZMTJLB2SERT9")
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&limit=1000&apikey=MLPIDKVUZKO71B19"
    else:
        if end_date:
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&time_to={end_date}T0130&limit=1000&apikey=LPHO83KMASND8ZJJ"
        else:
            endpoint = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={stock}&time_from={start_date}T0130&limit=1000&apikey=47WWMCYJI8DEB3KR"
    response = requests.get(endpoint)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

def convert_to_dataframe(news_items):
    """
    Converts a list of news items into a Polars DataFrame and extracts the date.
    
    :param news_items: List of news items.
    :return: A Polars DataFrame with an added 'date' column.
    """
    news_items = news_items.get("feed", [])
    if news_items:
        df = pl.DataFrame(news_items)
        df = df.with_columns(pl.col("time_published").str.slice(0, 8).alias("date"))
        return df
    return pl.DataFrame()

def aggregate_news_data(stocks, start_date, end_date, turn):
    """
    Fetches and aggregates news data for a specified number of API calls.
     
    :param num_calls: Number of API calls to make.
    :param endpoint: The API endpoint URL.
    :param headers: Dictionary containing request headers.
    :param params: Dictionary containing query parameters.
    :return: A Polars DataFrame containing aggregated news data.
    """
    df_list = []
    for stock in stocks:
        # Fetch the news data
        news_data = fetch_news_data(stock, start_date, end_date, turn)
        
        if news_data:
            # Convert to DataFrame and append to the list
            df = convert_to_dataframe(news_data)
            if df.height != 0:
                df_list.append(df)

    # Concatenate all DataFrames into a single DataFrame
    return pl.concat(df_list, how='vertical') if df_list else pl.DataFrame()

In [11]:
stocks = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'META', 'BRK-A', 'LLY', 'AVGO']
news_data = aggregate_news_data(stocks, 20230101, 20230831, True)

In [12]:
news_data = news_data.to_pandas()
news_data

""


In [6]:
news_data.to_csv('news_data.csv', mode = 'a', index = False, header= False)